In [ ]:
def process_w2(s3_uri):

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "image": {
                    "format": "png",
                    "source": {
                        "s3Location": {
                            "uri": s3_uri,
                            "bucketOwner" : account_id
                        }
                    }
                    }
                },
                {
                    "text": text_prompt
                }
            ]
        },
        {
            "role": "assistant",
            "content": [
                {"text": "```json"}
            ]
        }
    ]
    return messages

In [ ]:
import json

test_path = "data/merged_data/v1/nova_sft_testset.jsonl"

test_data = []
with open(test_path, 'r') as f:
    for line in f:
        if line.strip():
            sample = json.loads(line.strip())
            
            test_data.append()

print(f"num: {len(test_data)}")
print(f"example: {json.dumps(test_data[0], indent=2, ensure_ascii=False)}")

num: 1000
example: {
  "schemaVersion": "bedrock-conversation-2024",
  "system": [
    {
      "text": "##ROLE##\nYou are an advanced image classification specialist analyzing e-commerce product images to identify multiple relevant category tags from the ##REFERENCE_CATEGORIES##. Your primary focus is comprehensive coverage (recall) over strict precision.\n\n##PRIMARY_TASK##\nAnalyze the provided image and identify **UP TO 5 MOST RELEVANT** category tags from ##REFERENCE_CATEGORIES##. You are **STRONGLY ENCOURAGED** to output multiple tags when the image could reasonably belong to several categories or when there's reasonable uncertainty.\n\n##CRITICAL_MULTI_LABEL_REQUIREMENTS##\n\n**IMPORTANT: Multiple tags Are Expected and Preferred**\n- **Output 1-3 tags** whenever multiple categories from ##REFERENCE_CATEGORIES## are plausible\n- Think of this as providing comprehensive classification alternatives, not just the single best match\n- Consider related categories, different levels of s

In [ ]:

bedrock_client = boto3.client("bedrock-runtime", region_name="us-east-1")
deployment_arn = "arn:aws:bedrock:us-east-1:452145973879:custom-model-deployment/u83i7ozdfnqa"


response = bedrock_client.converse(
            modelId=deployment_arn,
            messages=process_w2(test_s3_paths[0].get("s3_uri")),
            inferenceConfig={"maxTokens": 2048, "temperature": 0.0, "topP": 0.1, "stopSequences": ["```"]},
        )

In [ ]:
prediction = json.loads(response["output"]["message"]["content"][0]["text"].replace("```", ""))
prediction